# Case Study 1 : Data Science in Twitter Data

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://cdn.oreillystatic.com/oreilly/booksamplers/9781449367619_sampler.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)
* [TED Talks](https://www.ted.com/talks) for examples of 10 minutes talks.


** NOTE **
* Please don't forget to save the notebook frequently when working in Jupyter Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem: pick a data science problem that you plan to solve using Twitter Data
* The problem should be important and interesting, which has a potential impact in some area.
* The problem should be solvable using twitter data and data science solutions.

Please briefly describe in the following cell: what problem are you trying to solve? why this problem is important and interesting?

We are looking at the trending hashtags and finding out the emotional responses people have in various regions due to 
those hashtags. We can categorize regions of the country by the emotional words used in tweets. This problem is interesting
because you can find out how regions feel about specific issues and trends by the way people react to them on twitter. This is
important because the data could be used by companies for advertising or politicians to find out how people feel about certain social
issues.

## Data Collection: Download Twitter Data using API

* In order to solve the above problem, you need to collect some twitter data. You could select a topic that is relevant to your problem, and use Twitter API to download the relevant tweets. It is recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [ ]:
import twitter, json, operator, sys
import collections
import numpy as np

#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'htBMGArjlGDW8FXLlb9iuXHW4'
    CONSUMER_SECRET ='udmNO1AFXMi1B5ZEmlmn0jY8oudyKkinXsa3ZpjzfMUwP0Txqj'
    OAUTH_TOKEN = '361006898-N3gjDZDTEvM6s4SQPNZGuJJN1e7HSGcnuHKr578l'
    OAUTH_TOKEN_SECRET = '0BNDtw48dwzu3PyWRd0zh2jMP977xGIy05rIQ1GAbQNJt'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
twitter_api = oauth_login()
query = '#GorsuchHearing' #Popular right now
question = {}
counter1 = 0
lowestID = 0
#<insert your code here>
#   Please add comments to explain the general idea of each block of the code.
for i in range (1, 3):
    if i == 1:
        tempQuestion = twitter_api.search.tweets(q=query, count=100)
    else:
        tempQuestion = twitter_api.search.tweets(q=query, count=100, max_id=lowestID)
    #didn't quite get this to build properly, got 100 tweets
    question = dict(question.items() + tempQuestion.items())
    for result in question["statuses"]:
        if result['id'] < lowestID or lowestID == 0:
            lowestID = result['id']

for result in question["statuses"]:
     counter1 += 1
     # print "(%s) @%s %s" % (result["created_at"], result["user"]["screen_name"], result["text"])


json_file_name = 'Results.json'
#---------
list_tweets = question
print list_tweets

#------------

with open(json_file_name, 'w') as outfile: #output the JSON in a dump, just the statuses
    json.dump(list_tweets["statuses"], outfile)


### Report  statistics about the tweets you collected 

In [ ]:
number_tweets = counter1
print 'Total acquired tweets: {}'.format(number_tweets)

# Data Exploration: Exploring the Tweets and Tweet Entities

**(1) Word Count:** 
* Load the tweets you collected in the local file (txt or json)
* compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 most-frequent words with their counts

In [ ]:
import json, operator
from prettytable import PrettyTable

json_file_name = 'Results.json'

# Load JSON file of tweets
result = []
new_list_of_tweets = json.loads(open(json_file_name, "r").read())

wordDict = dict();
for i in range (0, counter1):
    # Get text of tweet
    tempList = new_list_of_tweets[i]["text"].lower().split()
    
    # Add words to list
    for word in tempList:
        if word in wordDict:
            wordDict[word] += 1
        else:
            wordDict[word] = 1

table = PrettyTable(['Word', 'Count'])

# Sort words
sorted_wordDict = list(reversed(sorted(wordDict.items(), key=operator.itemgetter(1))))
tempCounter = 0

# Add rows to table
while tempCounter < 30:
    scrubbed_word = sorted_wordDict[tempCounter][0]  
    table.add_row([scrubbed_word,sorted_wordDict[tempCounter][1]])
    tempCounter += 1

print table

** (2) Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 most-retweeted tweets in your collection, i.e., the tweets with the largest number of retweet counts.


In [ ]:
# Get retweeted tweets
Associations = dict()
for i in range (0, counter1):
    Associations[new_list_of_tweets[i]["text"]] = new_list_of_tweets[i]["retweet_count"]

# Sort by count
sorted_Associations = list(reversed(sorted(Associations.items(), key=operator.itemgetter(1))))
tempCounter = 0

# Print top 10
while tempCounter < 10:
    print sorted_Associations[tempCounter][0].encode('utf-8').strip() + ": " + str(sorted_Associations[tempCounter][1])
    tempCounter += 1

**(3) Find the most popular Tweet Entities in your collection of tweets**

Please plot the top 10 most-frequent hashtags and top 10 most-mentioned users in your collection of tweets.

In [ ]:
import matplotlib.pyplot as plt

hashtags = dict()
mentioned_users = dict()

# Iterate through tweets
for i in range (0, number_tweets):
    # Get hashtags entity
    curr_hashtags = new_list_of_tweets[i]["entities"]["hashtags"]
    
    # Get mentions entity
    curr_mentions = new_list_of_tweets[i]["entities"]["user_mentions"]    
    
    # Get hashtag counts
    for j in range(0, len(curr_hashtags)):
        hashtag_text = curr_hashtags[j]["text"]
        if hashtag_text in hashtags:
            hashtags[hashtag_text] += 1
        else:
            hashtags[hashtag_text] = 1
            
    # Get mentions counts
    for j in range(0, len(curr_mentions)):
        mention_text = curr_mentions[j]["screen_name"]
        if mention_text in mentioned_users:
            mentioned_users[mention_text] += 1
        else:
            mentioned_users[mention_text] = 1

# Sort entities
sorted_hashtags = list(reversed(sorted(hashtags.items(), key=operator.itemgetter(1))))
sorted_mentioned_users = list(reversed(sorted(mentioned_users.items(), key=operator.itemgetter(1))))

# Get min of entity count and 10
hashtag_item_count = len(sorted_hashtags) if len(sorted_hashtags) < 10 else 10
mentioned_users_item_count = len(sorted_mentioned_users) if len(sorted_mentioned_users) < 10 else 10

# Hashtags graph
hashtags_texts = [hash[0] for hash in sorted_hashtags[0:10]]
hashtags_counts = [hash[1] for hash in sorted_hashtags[0:10]]

# Get colors
cmap = plt.get_cmap('gnuplot')
colors = [cmap(i) for i in np.linspace(0, 1, len(hashtags_texts))]

hashtags_len = range(len(hashtags_texts))
plt.bar(hashtags_len, hashtags_counts, align='center', color=colors)
locs, labels = plt.xticks(hashtags_len, hashtags_texts)
plt.setp(labels, rotation=90)
plt.title('Histogram of Hashtags')
plt.show()

# User mentions graph
user_mention_texts = [hash[0] for hash in sorted_mentioned_users[0:10]]
user_mention_counts = [hash[1] for hash in sorted_mentioned_users[0:10]]

colors = [cmap(i) for i in np.linspace(0, 1, len(user_mention_texts))]

user_mention_len = range(len(user_mention_texts))
plt.bar(user_mention_len, user_mention_counts, align='center', color=colors)
locs, labels = plt.xticks(user_mention_len, user_mention_texts)
plt.setp(labels, rotation=90)
plt.title('Histogram of User Mentions')
plt.show()

Plot a histogram of the number of user mentions in the list using the following bins.

In [ ]:
bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
bins_dict = dict()

# Initialize bins
for bin in bins:
    bins_dict[bin] = 0

# Put values in bins
for user in sorted_mentioned_users:
    count = user[1]
    bin_count = (count / 10) * 10
    
    bins_dict[bin_count] += 1

# Sort bins
sorted_bins = list((sorted(bins_dict.items(), key=operator.itemgetter(0))))

# Graph histogram
bin_ids = [bin[0] for bin in sorted_bins]
bin_vals = [bin[1] for bin in sorted_bins]

colors = [cmap(i) for i in np.linspace(0, 1, len(bin_ids))]

bins_len = range(len(bin_ids))
plt.bar(bins_len, bin_vals, align='center', color=colors)
locs, labels = plt.xticks(bins_len, bin_ids)
plt.title('Histogram of Frequencies of User Mentions')
plt.show()


 ** (4) Getting "All" friends and "All" followers of a popular user in the tweets**

* choose a popular twitter user who has many followers in your collection of tweets.
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
import time
import tweepy
from prettytable import PrettyTable

CONSUMER_KEY1 = 'htBMGArjlGDW8FXLlb9iuXHW4'
CONSUMER_SECRET1 ='udmNO1AFXMi1B5ZEmlmn0jY8oudyKkinXsa3ZpjzfMUwP0Txqj'
OAUTH_TOKEN1 = '361006898-N3gjDZDTEvM6s4SQPNZGuJJN1e7HSGcnuHKr578l'
OAUTH_TOKEN_SECRET1 = '0BNDtw48dwzu3PyWRd0zh2jMP977xGIy05rIQ1GAbQNJt'
auth = tweepy.OAuthHandler(CONSUMER_KEY1,CONSUMER_SECRET1)
auth.set_access_token(OAUTH_TOKEN1,OAUTH_TOKEN_SECRET1)
tweepyApi = tweepy.API(auth)
ids = []

# Get IDS of followers
for name in tweepy.Cursor(tweepyApi.followers_ids, screen_name="funder").items(20):
    ids.append(name)

# Get usernames from follower ids
screen_names = [user.screen_name for user in tweepyApi.lookup_users(user_ids=ids)]

# Print max 20 followers usernames and IDs
t = PrettyTable(['Follower UserName', 'ID'])
sizemax = 20

if len(screen_names)<20:
    sizemax = len(screen_names)
for x in range(0,sizemax):
    t.add_row([screen_names[x],ids[x]])
print t

# Get IDS of friends
ids2 = []
for name in tweepy.Cursor(tweepyApi.friends_ids, screen_name="funder").items(20):
    ids2.append(name)

# Get usernames from friend ids
screen_names2 = [user.screen_name for user in tweepyApi.lookup_users(user_ids=ids2)]

# Print max 20 followers usernames and IDs
t2 = PrettyTable(['Friend UserName', 'ID'])
sizemax = 20

if len(screen_names)<20:
    sizemax = len(screen_names)
for x in range(0,sizemax):
    t2.add_row([screen_names2[x],ids2[x]])

print t2

# The Solution: implement a data science solution to the problem you are trying to solve.

Briefly describe the idea of your solution to the problem in the following cell:

Our solution is taking twitter data from specific hashtags and looking for specific words and hashtags in those tweets. We will take twitter location data from each of those tweets and plot regions 

Write codes to implement the solution in python:

In [ ]:
# Status,hashtags,location
class parsedTweet:
    def __init__(self, status, hashtags, location):
        self.status = status
        self.hashtags = hashtags
        self.location = location

# Not 100% perfect, but good enough, can't code for every single way people write state information
def replaceState(text):
    statesDIC = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
    }
    
    ## not exactly in alphabetical order to avoid some replacement conflicts 
    
    states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho', 
         'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
         'Maine' 'Maryland','Massachusetts','Michigan','Minnesota',
         'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
         'New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio',    
         'Oklahoma','Oregon','Pennsylvania','Rhode Island',
         'South Carolina','South Dakota','Tennessee','Texas','Utah',
         'Vermont','Washington','West Virginia','Virginia',
         'Wisconsin','Wyoming']
    
    words = [x.strip() for x in text.split(',')]
    replaced_string = ' '.join(str(statesDIC.get(word,word)) for word in words)
    
    finalString = 'NONE'
    for statename in states:
        if replaced_string.lower().find(statename.lower()) != -1:
            finalString = statename
            break
            
    return finalString

# use this one for later on  
parsedTweetList = []

# this one is only for testing table printout
parsedTweetListtemp= []

# parse out the data we want to parse
for result in question["statuses"]:
    hashtags = []
    for text in result["entities"].get("hashtags"):
        hashtags.append(text["text"])
    tweet = parsedTweet(result["text"],hashtags,replaceState(result["user"]["location"]))
    tweet2 = parsedTweet(result["text"],hashtags,result["user"]["location"])
    parsedTweetList.append(tweet)
    parsedTweetListtemp.append(tweet)
    parsedTweetListtemp.append(tweet2)


## table to  show that we collected data correctly and replaced state names correctyl
pt = PrettyTable(['Status','Hashtag', 'Location'])

for parsed in parsedTweetListtemp:
    pt.add_row([parsed.status[0:10],parsed.hashtags[0:3],parsed.location])

print pt

In [ ]:
# Temporary state data, to be replaced by real data
state_data = {
    'Alaska': [{'sentiment': 'happy', 'count': 12}, {'sentiment': 'sad', 'count': 41}, {'sentiment': 'angry', 'count': 31}],
    'Alabama': [{'sentiment': 'happy', 'count': 22}, {'sentiment': 'sad', 'count': 2}, {'sentiment': 'angry', 'count': 12}],
    'Arkansas': [{'sentiment': 'happy', 'count': 15}, {'sentiment': 'sad', 'count': 75}, {'sentiment': 'angry', 'count': 54}],
    'Arizona': [{'sentiment': 'happy', 'count': 61}, {'sentiment': 'sad', 'count': 5}, {'sentiment': 'angry', 'count': 32}],
    'California': [{'sentiment': 'happy', 'count': 19}, {'sentiment': 'sad', 'count': 66}, {'sentiment': 'angry', 'count': 12}],
    'Colorado': [{'sentiment': 'happy', 'count': 2}, {'sentiment': 'sad', 'count': 4}, {'sentiment': 'angry', 'count': 21}]
}

# Select state for histogram
select_state = 'Arkansas'

# Sort by count
# sorted_sentiments = list(reversed(sorted(state_data[select_state], key=operator.itemgetter('count'))))
sorted_sentiments = state_data[select_state]

# Graph sentiment histogram
sentiments = [sent['sentiment'] for sent in sorted_sentiments]
sentiment_counts = [sent['count'] for sent in sorted_sentiments]

# Get colors
cmap = plt.get_cmap('gnuplot')
colors = [cmap(i) for i in np.linspace(0, 1, len(sentiments))]
sentiment_len = range(len(sentiments))

# Plot bars
plt.bar(sentiment_len, sentiment_counts, align='center', color=colors)
locs, labels = plt.xticks(sentiment_len, sentiments)
plt.setp(labels, rotation=90)
plt.title('Histogram of Sentiments in {}'.format(select_state))
plt.show()

In [ ]:
# For the map, we use the Basemap package, which can be found 
# here: https://sourceforge.net/projects/matplotlib/files/matplotlib-toolkits/basemap-1.0.7/

# We also require the shape files for the United States, which can be found
# here: https://github.com/matplotlib/basemap/tree/master/examples

# We base the map implementation from an example, which can be found
# here: http://stackoverflow.com/questions/7586384/color-states-with-pythons-matplotlib-basemap

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.patches import Rectangle
import sys

# Get first state for each of the sentiments
first_state = state_data[state_data.keys()[0]]

# Get colors
cmap = plt.get_cmap('gnuplot')
colors = {}
cmap_colors = [cmap(i) for i in np.linspace(0, 1, len(first_state))]

# Make dictionary of sentiments to colors
for i in range(len(first_state)):
    colors[first_state[i]['sentiment']] = cmap_colors[i]

def get_highest_sentinment(state_info):
    """
    Returns the sentiment with the highest count.
    
    Arguments:
        state_info: The array of sentiments and counts for the state.
    Returns:
        the sentiment
    """
    max_count = -sys.maxint - 1
    max_sentiment = None
    
    for i in range(len(state_info)):
        if state_info[i]['count'] >= max_count:
            max_count = state_info[i]['count']
            max_sentiment = state_info[i]['sentiment']
    
    return max_sentiment

# Build map
map = Basemap(llcrnrlon=-119, llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49, projection='lcc', lat_1=33, lat_2=45, lon_0=-95)

# load shapefile
map.readshapefile('map_data/st99_d00', name='states', drawbounds=True)

# Collect names of states for lookup table
state_names = []
for shape_dict in map.states_info:
    state_names.append(shape_dict['NAME'])

# Axes instance
ax = plt.gca()

# Plot polygon of each state
for state in state_data:
    color = colors[get_highest_sentinment(state_data[state])]
    seg = map.states[state_names.index(state)]
    poly = Polygon(seg, facecolor=color,edgecolor='black')
    ax.add_patch(poly)

# Create legend of sentiments
legend_rects = []
legend_texts = []

# Add color and name to legend
for i in range(len(colors.keys())):
    sentiment = colors.keys()[i]
    legend_rects.append(Rectangle((0, 0), 1, 1, fc=colors[sentiment], fill=True, edgecolor='none', linewidth=0))
    legend_texts.append(sentiment)

ax.legend(legend_rects, legend_texts, loc=2, bbox_to_anchor=(1.0, 0.7))
plt.show()

# Results: summarize and visualize the results discovered from the analysis

Please use figures, tables, or videos to communicate the results with the audience.


In [ ]:
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary








*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this Jupyter notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "jupyter notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.

* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . Each team present their case studies in class for 10 minutes.

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through Canvas, in the Assignment "Case Study 1".
        
** Note: Each team only needs to submit one submission in Canvas **


# Peer-Review Grading Template:

** Total Points: (100 points) ** Please don't worry about the absolute scores, we will rescale the final grading according to the performance of all teams in the class.

Please add an "**X**" mark in front of your rating: 

For example:

*2: bad*
          
**X** *3: good*
    
*4: perfect*


    ---------------------------------
    The Problem: 
    ---------------------------------
    
    1. (5 points) how well did the team describe the problem they are trying to solve using twitter data? 
       0: not clear
       1: I can barely understand the problem
       2: okay, can be improved
       3: good, but can be improved
       4: very good
       5: crystal clear
    
    2. (10 points) do you think the problem is important or has a potential impact?
        0: not important at all
        2: not sure if it is important
        4: seems important, but not clear
        6: interesting problem
        8: an important problem, which I want to know the answer myself
       10: very important, I would be happy invest money on a project like this.
    
    ----------------------------------
    Data Collection:
    ----------------------------------
    
    3. (10 points) Do you think the data collected are relevant and sufficient for solving the above problem? 
       0: not clear
       2: I can barely understand what data they are trying to collect
       4: I can barely understand why the data is relevant to the problem
       6: the data are relevant to the problem, but better data can be collected
       8: the data collected are relevant and at a proper scale (> 300 tweets)
      10: the data are properly collected and they are sufficient

    -----------------------------------
    Data Exploration:
    -----------------------------------
    4. How well did the team solve the following task:
    (1) Word Count (5 points):
       0: missing answer
       1: okay, but with major problems
       3: good, but with minor problems
       5: perfect
    
    (2) Find the most popular tweets in your collection of tweets: (5 points)
       0: missing answer
       1: okay, but with major problems
       3: good, but with minor problems
       5: perfect
    
    (3) Find popular twitter entities  (5 points)
       0: missing answer
       1: okay, but with major problems
       3: good, but with minor problems
       5: perfect

    (4) Find user's followers and friends (5 points)
       0: missing answer
       1: okay, but with major problems
       3: good, but with minor problems
       5: perfect

    -----------------------------------
    The Solution
    -----------------------------------
    5.  how well did the team describe the solution they used to solve the problem? 
       0: not clear
       2: I can barely understand
       4: okay, can be improved
       6: good, but can be improved
       8: very good
       10: crystal clear
       
    6. how well is the solution in solving the problem? 
       0: not relevant
       1: barely relevant to the problem
       2: okay solution, but there is an easier solution.
       3: good, but can be improved
       4: very good, but solution is simple/old
       5: innovative and technically sound
       
    7. how well did the team implement the solution in python? 
       0: the code is not relevant to the solution proposed
       2: the code is barely understandable, but not relevant
       4: okay, the code is clear but incorrect
       6: good, the code is correct, but with major errors
       8: very good, the code is correct, but with minor errors
      10: perfect 
   
    -----------------------------------
    The Results
    -----------------------------------
     8.  How well did the team present the results they found in the data? 
       0: not clear
       2: I can barely understand
       4: okay, can be improved
       6: good, but can be improved
       8: very good
      10: crystal clear
       
     9.  How do you think the results they found in the data? 
       0: not clear
       1: likely to be wrong
       2: okay, maybe wrong
       3: good, but can be improved
       4: make sense, but not interesting
       5: make sense and very interesting
     
    -----------------------------------
    The Presentation
    -----------------------------------
    10. How all the different parts (data, problem, solution, result) fit together as a coherent story?  
       0: they are irrelevant
       1: I can barely understand how they are related to each other
       2: okay, the problem is good, but the solution doesn't match well, or the problem is not solvable.
       3: good, but the results don't make much sense in the context
       4: very good fit, but not exciting (the storyline can be improved/polished)
       5: a perfect story
      
    11. Did the presenter make good use of the 10 minutes for presentation?  
       0: the team didn't present
       1: bad, barely finished a small part of the talk
       2: okay, barely finished most parts of the talk.
       3: good, finished all parts of the talk, but some part is rushed
       4: very good, but the allocation of time on different parts can be improved.
       5: perfect timing and good use of time      

    12. How well do you think of the presentation (overall quality)?  
       0: the team didn't present
       1: bad
       2: okay
       3: good
       4: very good
       5: perfect


    -----------------------------------
    Overall: 
    -----------------------------------
    13. How many points out of the 100 do you give to this project in total?  Please don't worry about the absolute scores, we will rescale the final grading according to the performance of all teams in the class.
    Total score:
    
    14. What are the strengths of this project? Briefly, list up to 3 strengths.
       1: 
       2:
       3:
    
    15. What are the weaknesses of this project? Briefly, list up to 3 weaknesses.
       1:
       2:
       3:
    
    16. Detailed comments and suggestions. What suggestions do you have for this project to improve its quality further.
    
    
    

    ---------------------------------
    Your Vote: 
    ---------------------------------
    1. [Overall Quality] Between the two submissions that you are reviewing, which team would you vote for a better score?  
       -1: I vote the other team is better than this team
        0: the same
        1: I vote this team is better than the other team 
        
    2. [Presentation] Among all the teams in the presentation, which team do you think deserves the best presentation award for this case study?  
        1: Team 1
        2: Team 2
        3: Team 3
        4: Team 4
        5: Team 5
        6: Team 6
        7: Team 7
        8: Team 8
        9: Team 9
       10: Team 10

